In [1]:
from bs4 import BeautifulSoup as bs
import requests as rq
import csv 
import pandas as pd

In [2]:
def books_links(link):
    categories={}
    response=rq.request('GET',link)
    soup=bs(response.content,'html.parser')
    all_links=soup.find('div',attrs={'class':'side_categories'}).find('ul').find('li').find('ul').find_all('a')
    for i in range(0,len(all_links)):
        url=all_links[i].get('href').replace('../','http://books.toscrape.com/catalogue/category/')
        if url != 'index.html':
            category_name=all_links[i].get_text().strip()
            categories[category_name]=url
    return categories

In [3]:
categories=books_links('http://books.toscrape.com/catalogue/category/books_1/index.html')

In [4]:
for category_name , url in categories.items():
    category_name=category_name
    url=url
    with open('all_books.csv','a+') as f:
        fnames=['Names','Rates','Prices','category_name','url']
        writer=csv.DictWriter(f,fieldnames=fnames)
        writer.writeheader()
        rate={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}
        response=rq.request('GET',url)
        soup=bs(response.content,'html.parser')
        all_articles=soup.find_all('article',attrs={'class':'product_pod'})

        for article in all_articles: 
            ratting=article.find_all('p')[0].attrs['class'][1]
            rates=rate[ratting]
            prices=float(article.find_all('p')[1].get_text().split('£')[1])
            names=article.h3.a.get_text()
            data={'Names':names,'Rates':rates,'Prices':prices,'category_name':category_name,'url':url}
            writer.writerow(data)
            #print(f'{names}\n{rates}\n{prices}\n{category_name}\n{url}')


In [6]:
df=pd.read_csv('all_books.csv',encoding='cp1252')
df

,Names,Rates,Prices,category_name,url
0,It's Only the Himalayas,2,45.17,Travel,http://books.toscrape.com/catalogue/category/b...
1,Full Moon over Noah’s ...,4,49.43,Travel,http://books.toscrape.com/catalogue/category/b...
2,See America: A Celebration ...,3,48.87,Travel,http://books.toscrape.com/catalogue/category/b...
3,Vagabonding: An Uncommon Guide ...,2,36.94,Travel,http://books.toscrape.com/catalogue/category/b...
4,Under the Tuscan Sun,3,37.33,Travel,http://books.toscrape.com/catalogue/category/b...
...,...,...,...,...,...
3397,Amid the Chaos,1,36.58,Cultural,http://books.toscrape.com/catalogue/category/b...
3398,Names,Rates,Prices,category_name,url
3399,Dark Notes,5,19.19,Erotica,http://books.toscrape.com/catalogue/category/b...
3400,Names,Rates,Prices,category_name,url


In [7]:
df.describe()

,Names,Rates,Prices,category_name,url
count,3402,3402,3402,3402,3402
unique,516,6,498,51,51
top,Names,1,Prices,Crime,url
freq,300,732,300,522,300
